# Packages
You can use the `#r` statement to import packages from nuget. It is possible to omit the package version but specifying a version prevents future breakage by package updates. If you change versions or if the package is updated when you haven't specified a version, you will need to restart the kernel.

In [ ]:
#r "nuget: Plotly.NET, 3.0.1"
#r "nuget: Plotly.NET.Interactive, 3.0.1"
#r "nuget: CsvHelper, 30.0.1"

Installing Packages CsvHelper Plotly.NET Plotly.NET.Interactive

# Imports
You can specify using statements just as you would in C#. A few common namespaces like `System` and `System.Text` are imported by default.

In [ ]:
using System.IO;
using Plotly.NET;
using CsvHelper;

# Data
Now you can use your favourite .NET tooling to get some data. In this example, we'll use `CsvHelper` to read a CSV file containing all FIFA world cup matches until 2018.

In [ ]:
using System.IO;
using System.Globalization;
using Plotly.NET;
using CsvHelper;

public class Match {
    public int Year;
}

private List<Match> LoadData() {
    using(var reader = new StreamReader("data/matches_1930_2018.csv")) 
    using(var csv = new CsvReader(reader, CultureInfo.InvariantCulture))
    {
        return csv.GetRecords<Match>().ToList();
    }
}

var data = LoadData();
Console.WriteLine($"Loaded {data.Count} matches");


Error: (11,43): error CS0103: The name 'CultureInfo' does not exist in the current context